# 增强--对整个数据集进行平衡增强

In [8]:
from pyecharts.globals import CurrentConfig, NotebookType, OnlineHostType
CurrentConfig.ONLINE_HOST = "https://assets.pyecharts.org/assets/"
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
from IPython.display import display, clear_output
import os
import datetime
import time
import csv
import sympy
import pandas as pd
from pyecharts.charts import Bar,Grid,Pie,Line
from pyecharts import options as opts
from class_aug import directory_check,get_files,class_num,log_file,if_balanced,class_sort,image_sort,get_aug_list,\
                       augmentation,draw,draw_show,goal_class,find_class,if_satisfy,log_file,round_time,class_percent,\
                       pyecharts_line_bar,pyecharts_pie,pyecharts_bar_stack

In [9]:
# augmentation class numbers every cycle

AUG_NUMBER = 2

# the final dataset's distance among every classes

PERCENT = 0.01

# the path of source annotations and images

xml_source_path = 'D:/lab/testpack/augtest/xml/'
image_source_path = 'D:/lab/testpack/augtest/jpg/'

# the path to save new annotations and images

xml_path = 'D:/lab/testpack/augtest/augmentation_5/xml/'
image_path = 'D:/lab/testpack/augtest/augmentation_5/jpg/'

# the log filepath

log_path = os.path.join(xml_path, os.path.pardir+"/"+"log.csv")

## 对原数据集里的数据进行统计、展示

In [10]:
filenames = get_files(xml_source_path)
source_class_names, source_num_objs, source_source_list = class_num(filenames,xml_source_path)
source_file_name = list(source_source_list.keys())
x_data = list(source_num_objs.keys())
v1 = list(source_num_objs.values())
average,average_list = class_percent(source_num_objs)
v2 = [i*100 for i in average_list]
grid = pyecharts_line_bar(x_data, v1, v2, source_num_objs)
pie = pyecharts_pie(x_data, v1)
grid.render_notebook()
pie.render_notebook()
df = pd.DataFrame([list(source_num_objs.keys()),list(source_num_objs.values())])
pd.set_option('display.max_columns',200)
df


,0,1,2,3,4,5,6,7
0,plastic,phone,bottle,mirror,umbrella,laggage,knife,scissors
1,1,1,2,1,1,2,1,1


## 开始增强 

In [ ]:
directory_check(xml_path,image_path,xml_source_path,image_source_path)
times = 0
a = 0
start_time = datetime.datetime.now()
# get and save the source data of the dataset
filenames = get_files(xml_path)
source_class_names, source_num_objs, source_source_list = class_num(filenames, xml_path)
source_file_name = list(source_source_list.keys())
# if dataset not balanced, continue the cycle
while a == 0:
    clear_output(wait=True)
    filenames = get_files(xml_path)
    # get quantity of every class
    class_names, num_objs, source_list = class_num(filenames, xml_path)
    # write file log
    log_file(times, source_class_names, num_objs, log_path)
    # judge if the quantity of every class is balanced
    a = if_balanced(num_objs, times, source_list, PERCENT)
    if a == 0:
        # sort every class
        class_list = class_sort(num_objs)
        # sort every picture
        image_list = image_sort(class_list, source_source_list)
        # get the final augmentation list
        aug_image_list = get_aug_list(image_list, class_list, AUG_NUMBER)
        # augmentation
        print("round:", times)
        augmentation(aug_image_list, times, source_file_name, xml_path, image_path)
        times = times + 1
        #------------------画图------------------------------------
        v1 = list(source_num_objs.values())
        average,average_list = class_percent(source_num_objs)
        v2 = [i*100 for i in average_list]
        grid = pyecharts_line_bar(x_data, v1, v2, source_num_objs)
        grid.render_notebook()
        #----------------画图--------------------------------------
    else:
        print('finish augmentation')
#         draw(source_num_objs,num_objs, xml_path)
        end_time = datetime.datetime.now()
        spend_time = (end_time - start_time).seconds
        print("time spend: ", spend_time/60, "min")
        with open(log_path, 'a+', newline='') as f:
            csv_write = csv.writer(f)
            data_row = ['time_spend', spend_time/60, 'min']
            csv_write.writerow(data_row)
        break
    time.sleep(2)
#--------------------画图----------------------------------   
v1 = list(source_num_objs.values())
average,average_list = class_percent(source_num_objs)
v2 = [i*100 for i in average_list]
grid = pyecharts_line_bar(x_data, v1, v2, source_num_objs)
pie = pyecharts_pie(x_data, v1)
grid.render_notebook()
pie.render_notebook()
#--------------------画图---------------------------------- 